## r/AmItheAsshole Lanugage Analysis

### Introduction

[r/AmItheAsshole](https://www.reddit.com/r/AmItheAsshole/) is a subreddit where people will post a story about some conflict they've had and it is up to the community to judge who the "asshole" is in the situation. The page description words this very eloquently:

"A catharsis for the frustrated moral philosopher in all of us, and a place to finally find out if you were wrong in an argument that's been bothering you. Tell us about any non-violent conflict you have experienced; give us both sides of the story, and find out if you're right, or you're the asshole." ([r/AmItheAsshole](https://www.reddit.com/r/AmItheAsshole/))

The goal of this project if there are any notable lexical features between the langauge of the "assholes" and those who are not.

One thing that I would like to note is that this is not a corpus directly translated from speech so the language of the mosts might have been thoughfully worded and might not be representative of ones's natural speech. Still I believe that the judgments voted on by the reddit community can provide insight on.

### Scraping from Reddit
#### PRAW
[The Python Reddit API Wrapper (PRAW)](https://praw.readthedocs.io/en/v4.1.0/index.html) is an API for anything you can do in Reddit. In this project it is used to scrape posts from the r/AmItheAsshole subreddit.

#### The r/AmItheAsshole Archives
The subreddit archives the posts into 4 different categories related to this project. There are 5 judgements that one can decide on, however the last one that is not archived 'INFO' (Not enough info) does not help with this investigation. The different judgements are described in the function below.

#### Helper functions for scraping from reddit

In [2]:
import os
import glob
import yaml
import praw
import pandas as pd
from tqdm import tqdm_notebook



# Initialize reddit API
reddit = praw.Reddit('scraper', user_agent='corpus ling project')
reddit.read_only = True

# Abbreviations for the archive queries
archive_names = { 'TYA' : 'Asshole', 'NTA' : 'Not the A-hole', 'ESH' : 'Everyone Sucks',
        'NAH' : 'No A-holes here'}

# Pandas dataframe containing all of the posts.
corpus = pd.DataFrame()

def get_archive(judgment):
    '''
    Get the posts from a single archive of r/AmItheAsshole
    Archive judgement options are:
        'TYA' -- You're the Asshole (& the other party is not)
        'NTA' -- You're Not the A-hole (& the other party is)
        'ESH' -- Everyone Sucks Here
        'NAH' -- No A-holes here
    '''
    return reddit.subreddit('AmItheAsshole').search(
            'flair_name:"' + archive_names[judgment] + '"', limit=None)

def load_archives(download_local = True):
    '''
    Grabs each of the archives and puts them in the corpus dictionary.
    The download_local flag determines if you want to download the text
    as well in the current directory.
    '''
    global corpus 
    corpus = pd.DataFrame()
    for judgement in tqdm_notebook(archive_names.keys()):
        posts = get_archive(judgement)
        row_df = pd.io.json.json_normalize(
                map(lambda post : {'archive' : judgement, **vars(post)}, posts))
        # store only the id, title, archive, rawtext, and url
        corpus = corpus.append(row_df[['id', 'title', 'archive', 'selftext', 'url']],
                               ignore_index = True, sort=True)        
        
        # Download the text locally
        if (download_local):
            if not os.path.exists(judgement):
                # separate archives into different directories.
                os.mkdir(judgement)
            for index, post in (corpus.loc[corpus['archive'] == judgement]).iterrows():
                # Save each file in their respective directories with their id as the filename
                with open(judgement + '/' + post.id + '.yaml', 'w') as file:
                    documents = yaml.dump(post.to_dict(), file)

def local_load_archives():
    '''
    Loads the archived posts from yaml files in the current directory.
    '''
    global corpus 
    corpus = pd.DataFrame()
    for judgement in tqdm_notebook(archive_names.keys()):
        for filename in glob.glob(judgement + '/*.yaml'):
            with open(filename) as file:
                corpus = corpus.append(pd.io.json.json_normalize(
                    yaml.load(file, Loader=yaml.FullLoader)),
                                       ignore_index = True, sort=True)



#### Several options for loading the corpus of posts:
To keep things consistent, please use the `local_load_archives()` option becuase posts might be added over time and so the anaysis of this corpus may change as new posts are added.

Note: `load_archives()` will not work unless you have a `praw.ini` file in the current directory. This file contains the necesary account information for using the Reddit API.

In [ ]:
# Run this if you just want to use the posts that are downloaded locally.
local_load_archives()

In [ ]:
# Run this if you want the most recent set of posts but without downloading them locally.
load_archives(False)

In [ ]:
# Run this if you want to load and update the local files with the current archives.
load_archives()

#### The corpus
The corpus of posts is represented as a datarame whos most important columns are the archive column, and the text. The following code prints out summaries of all of the archives with the counts of posts.

In [4]:
for judgement in archive_names.keys():
    print("\n\n Summary of all of the posts in the \"" + archive_names[judgement] + "\" archive:")
    print(corpus.loc[corpus['archive'] == judgement].describe())



 Summary of all of the posts in the "Asshole" archive:
       archive      id                                           selftext  \
count      238     238                                                238   
unique       1     238                                                238   
top        TYA  e1w8za  So the other day, during a great evening with ...   
freq       238       1                                                  1   

                                         title  \
count                                      238   
unique                                     238   
top     AITA for not attending family therapy?   
freq                                         1   

                                                      url  
count                                                 238  
unique                                                238  
top     https://www.reddit.com/r/AmItheAsshole/comment...  
freq                                                    1  


 Sum

##### Feel free to take a break and read a random post:

In [5]:
post = corpus.sample(n=1)
print(post.selftext.tolist()[0])

My wife and I have been planning a two week family vacation to Europe, where most of my wife's extended family and friends live, for a long time now, and we have our trip booked for Dec. 7 through Dec. 22. This is supposed to be both our first big vacation as a family and the first time her relatives would get to meet our 3 year old daughter.

For some additional background, I am currently the sole breadwinner for our family, and my wife is a stay at home mom and cares for our daughter full time. I work for a small firm that consists of five people: my boss plus four employees he manages, including me. One of my coworkers is currently out on FMLA leave for medical issues and is tentatively scheduled to return in January if her recovery goes well.

In the past week, everything has basically gone to shit at the office. My two other coworkers just both put in their two weeks notice within days of each other, and we have a huge project due by the end of the year. The timing couldn't possib

##### Can you guess how the community judged this submission?

In [6]:
print(archive_names[post.archive.tolist()[0]])

Asshole


### Analyzing the Corpus
#### NLTK
[Natural Language Toolkit (NLTK)](https://www.nltk.org/) is a python library with a lot of Natural Language processing tools.

First we need to import nltk and download some tools:

In [ ]:
import nltk
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('vader_lexicon')

Some helper functions for using the nltk library on the corpus dataframe:

In [8]:
from nltk import word_tokenize, sent_tokenize, Text, FreqDist

def corpus_tokenize():
    '''
    Converts the text of each post to a list of tokens and places
    them in a list in a column within the corpus dataframe.
    '''
    global corpus
    corpus['tokens'] = list(map(lambda text : word_tokenize(text.lower()), corpus['selftext']))
    # Get rid of punctuation
    tokenizer = nltk.tokenize.RegexpTokenizer(r'\w+')
    corpus['tokens_no_punct'] = list(map(lambda text : tokenizer.tokenize(text.lower()), corpus['selftext']))
    # Tokenize on sentences.
    corpus['sent_tokens'] = list(map(sent_tokenize, corpus['selftext']))
    
def get_archive_texts(archives, no_punctuation = False):
    '''
    Gets the texts of every post of the archives listed in archives
    and returns it as one list of words.
    '''
    global corpus
    texts = []
    for archive in archives:
        for post_tokens in corpus.loc[corpus['archive'] == archive]['tokens_no_punct' if no_punctuation else 'tokens'].to_list():
            texts += post_tokens
    return texts

def get_archive_texts_sentences(archives):
    '''
    Gets the texts of every post of the archives listed in archives
    and returns it as one list of sentences.
    '''
    global corpus
    texts = []
    for archive in archives:
        for sent_tokens in corpus.loc[corpus['archive'] == archive]['sent_tokens'].to_list():
            texts += sent_tokens
    return texts

def get_freq_dist(archives, no_stopwords = False):
    '''
    Get a FreqDist for the given archive names There is also the option to remove stopwords.
    '''
    text = get_archive_texts(archives, no_punctuation=True)
    if no_stopwords:
        # filter out stopwords
        stopwords = nltk.corpus.stopwords.words('english')
        text = list(filter(lambda word : not word in stopwords, text))
    return FreqDist(text)
        

#### Preprocessing the Corpus
Creates three new columns in the corpus dataframe, 'tokens', 'tokens_no_punct', and 'sent_tokens'. One with just the raw output of the nltk tokenize function, the next without punctiation, and finally the text separated by sentences.

In [9]:
corpus_tokenize()
corpus.head()

,archive,id,selftext,title,url,tokens,tokens_no_punct,sent_tokens
0,TYA,e8l9h6,My husband (30M) is defending his PhD in 1 wee...,WIBTA if I didn't go to my husband's PhD defense,https://www.reddit.com/r/AmItheAsshole/comment...,"[my, husband, (, 30m, ), is, defending, his, p...","[my, husband, 30m, is, defending, his, phd, in...",[My husband (30M) is defending his PhD in 1 we...
1,TYA,duxns4,"So Thursday, I went to a party one of my frien...",AITA for being livid at my roommate for callin...,https://www.reddit.com/r/AmItheAsshole/comment...,"[so, thursday, ,, i, went, to, a, party, one, ...","[so, thursday, i, went, to, a, party, one, of,...","[So Thursday, I went to a party one of my frie..."
2,TYA,e617k1,I wanted to do something nice for my mom for h...,AITA for not painting my step father in a fami...,https://www.reddit.com/r/AmItheAsshole/comment...,"[i, wanted, to, do, something, nice, for, my, ...","[i, wanted, to, do, something, nice, for, my, ...",[I wanted to do something nice for my mom for ...
3,TYA,e8tpjf,Edit: threw OUT* a diaper beside my head\n\nTh...,AITA for being mad someone threw a diaper next...,https://www.reddit.com/r/AmItheAsshole/comment...,"[edit, :, threw, out*, a, diaper, beside, my, ...","[edit, threw, out, a, diaper, beside, my, head...",[Edit: threw OUT* a diaper beside my head\n\nT...
4,TYA,ebgst8,A little background. I met a guy (‘Mark’) at w...,WIBTA if I sent someone back their Christmas c...,https://www.reddit.com/r/AmItheAsshole/comment...,"[a, little, background, ., i, met, a, guy, (, ...","[a, little, background, i, met, a, guy, mark, ...","[A little background., I met a guy (‘Mark’) at..."


#### Word Frequencies
Since these posts are written in the first person and talk about personal experiences, a lot of the most frequent words are pronouns, therefore the following frequency counts remove stopwords. This can be changed by replacing `True` with `False` in the no_stopwords field.

In [10]:
pd.DataFrame(get_freq_dist(archive_names.keys(), no_stopwords=True).most_common(20), columns=['Word', 'Frequencey'])

,Word,Frequencey
0,like,1365
1,said,1303
2,would,1114
3,told,1086
4,time,1004
5,get,954
6,one,912
7,got,841
8,want,835
9,really,805


In [11]:
n_most_common = 50

pd.DataFrame(get_freq_dist(['TYA', 'ESH'], no_stopwords=True).most_common(n_most_common),
             columns=['word', 'frequencey']).join(
    pd.DataFrame(get_freq_dist(['NTA', 'NAH'], no_stopwords=True).most_common(n_most_common),
                 columns=['word', 'frequencey']), lsuffix="_a_hole")

,word_a_hole,frequencey_a_hole,word,frequencey
0,like,709,like,656
1,said,680,said,623
2,told,526,would,603
3,time,524,told,560
4,would,511,time,480
5,get,475,get,479
6,one,438,one,474
7,got,406,want,462
8,know,375,got,435
9,want,373,really,434


#### N-grams and Collocations
Lets explore making bigrams and trigrams and finding collocations for this corpus.

First lets look at the collocations generated from bigrams. There are several different scoring measures that can be used as is listed in one of the comments below.

In [17]:
import nltk
from nltk.collocations import *

# Feel free to change these values.
n_collocs = 50
n_freq_filter = 5

bigram_measures = nltk.collocations.BigramAssocMeasures()
# Options: pmi, likelihood_ratio, chi_sq, dice, phi_sq, etc.
scoring_measure = bigram_measures.pmi

flatten = lambda l: (l[0][0], l[0][1], l[1])

finder = BigramCollocationFinder.from_words(get_archive_texts(['TYA', 'ESH'], no_punctuation=True))
finder.apply_freq_filter(n_freq_filter)
a_hole_colloc = pd.DataFrame(list(map(flatten, finder.score_ngrams(scoring_measure))),
                             columns=['bigram_word_1', 'bigram_word_2', 'score'])

finder = BigramCollocationFinder.from_words(get_archive_texts(['NTA', 'NAH'], no_punctuation=True))
finder.apply_freq_filter(n_freq_filter)
colloc = pd.DataFrame(list(map(flatten, finder.score_ngrams(scoring_measure))),
                      columns=['bigram_word_1', 'bigram_word_2', 'score'])

a_hole_colloc.join(colloc, lsuffix="_a_hole").head(n_collocs)

,bigram_word_1_a_hole,bigram_word_2_a_hole,score_a_hole,bigram_word_1,bigram_word_2,score
0,mr,lastname,13.971120,baked,goods,14.290217
1,peanut,butter,13.878011,nail,polish,14.249575
2,passive,aggressive,13.685366,ice,cream,13.860533
3,fake,lashes,13.462973,tl,dr,13.605719
4,tl,dr,13.462973,imgur,com,13.342684
5,y,o,13.177571,plant,based,13.342684
6,400,calories,13.170192,passive,aggressive,13.283791
7,blah,blah,13.018189,cow,milk,13.227207
8,ice,cream,12.626472,https,imgur,13.190681
9,social,media,12.604992,co,worker,13.120292


What about trigrams?

In [18]:
import nltk
from nltk.collocations import *

# Feel free to change these values.
n_collocs = 50
n_freq_filter = 5

trigram_measures = nltk.collocations.TrigramAssocMeasures()
# Options: pmi, raw_freq, likelihood_ratio, chi_sq, jaccard, dice, phi_sq, etc.
scoring_measure = trigram_measures.pmi

flatten = lambda l: (l[0][0], l[0][1], l[0][2], l[1])

finder = TrigramCollocationFinder.from_words(get_archive_texts(['TYA', 'ESH'], no_punctuation=True))
finder.apply_freq_filter(n_freq_filter)
a_hole_colloc = pd.DataFrame(list(map(flatten, finder.score_ngrams(scoring_measure)[:n_collocs])),
                             columns=['trigram_word_1', 'trigram_word_2', 'trigram_word_3', 'score'])

finder = TrigramCollocationFinder.from_words(get_archive_texts(['NTA', 'NAH'], no_punctuation=True))
finder.apply_freq_filter(n_freq_filter)
colloc = pd.DataFrame(list(map(flatten, finder.score_ngrams(scoring_measure)[:n_collocs])),
                      columns=['trigram_word_1', 'trigram_word_2', 'trigram_word_3', 'score'])

a_hole_colloc.join(colloc, lsuffix="_a_hole")

,trigram_word_1_a_hole,trigram_word_2_a_hole,trigram_word_3_a_hole,score_a_hole,trigram_word_1,trigram_word_2,trigram_word_3,score
0,long,story,short,21.092718,https,imgur,com,26.533366
1,on,social,media,19.097141,less,active,side,21.100128
2,at,5,30am,17.829726,long,story,short,20.764079
3,my,mouth,shut,17.099538,blake,s,ring,18.137165
4,a,phd,student,16.955482,aunt,tina,s,17.906995
5,of,ice,cream,16.827380,my,mouth,shut,17.832325
6,few,days,ago,16.810970,the,self,feeder,17.642068
7,along,the,lines,16.776775,under,the,impression,16.965110
8,6,months,ago,16.518294,11,year,old,16.809501
9,fast,forward,to,16.389358,2,3,minutes,16.676940


#### Sentiment Analysis
NLTK provides a sentiment analysis model already called [VADER](https://github.com/cjhutto/vaderSentiment):

"VADER (Valence Aware Dictionary and sEntiment Reasoner) is a lexicon and rule-based sentiment analysis tool that is specifically attuned to sentiments expressed in social media."

The following are some functions to help process the text with the sentiment analyzer.

In [22]:
from nltk.sentiment.vader import SentimentIntensityAnalyzer

def get_average_sentiment_scores(text):
    '''
    Takes in a list of sentences to get the averge sentiment
    scores for each sentence using VADER.
    '''
    sid = SentimentIntensityAnalyzer()
    averages = {'neg': 0.0, 'neu': 0.0, 'pos': 0.0, 'compound': 0.0}
    count = 0
    for sentence in text:
        ss = sid.polarity_scores(sentence)
        for key, val in ss.items():
            averages[key] += val
        count += 1
    for key in averages.keys():
        averages[key] /= count
    return averages

def corpus_sentiment_scores():
    '''
    Updates the corpus with the average sentiment scores of each post.
    '''
    global corpus
    sid = SentimentIntensityAnalyzer()
    scores = list(map(get_average_sentiment_scores, corpus['sent_tokens']))
    corpus = corpus[corpus.columns.difference(scores[0].keys())].join(
        pd.DataFrame(list(map(lambda d : d.values(), scores)), columns=scores[0].keys()))
    
def average_archive_sentiment_scores(archives):
    '''
    Gets the average sentiment scores for each archive and returns it
    as a dataframe.
    '''
    global corpus
    sentiment_scores = corpus.loc[
        list(map(lambda row: row[1]['archive'] in archives,
                 list(corpus.iterrows())))]
    return sentiment_scores.mean(axis=0).to_dict()
    
    

Lets first put these scores into the corpus dataframe. 

In [20]:
# Preprocessing
corpus_sentiment_scores()
corpus.head()

,archive,id,selftext,sent_tokens,title,tokens,tokens_no_punct,url,neg,neu,pos,compound
0,TYA,e8l9h6,My husband (30M) is defending his PhD in 1 wee...,[My husband (30M) is defending his PhD in 1 we...,WIBTA if I didn't go to my husband's PhD defense,"[my, husband, (, 30m, ), is, defending, his, p...","[my, husband, 30m, is, defending, his, phd, in...",https://www.reddit.com/r/AmItheAsshole/comment...,0.074684,0.819947,0.105368,0.099732
1,TYA,duxns4,"So Thursday, I went to a party one of my frien...","[So Thursday, I went to a party one of my frie...",AITA for being livid at my roommate for callin...,"[so, thursday, ,, i, went, to, a, party, one, ...","[so, thursday, i, went, to, a, party, one, of,...",https://www.reddit.com/r/AmItheAsshole/comment...,0.073577,0.864731,0.061615,-0.061638
2,TYA,e617k1,I wanted to do something nice for my mom for h...,[I wanted to do something nice for my mom for ...,AITA for not painting my step father in a fami...,"[i, wanted, to, do, something, nice, for, my, ...","[i, wanted, to, do, something, nice, for, my, ...",https://www.reddit.com/r/AmItheAsshole/comment...,0.047857,0.851571,0.100571,0.095214
3,TYA,e8tpjf,Edit: threw OUT* a diaper beside my head\n\nTh...,[Edit: threw OUT* a diaper beside my head\n\nT...,AITA for being mad someone threw a diaper next...,"[edit, :, threw, out*, a, diaper, beside, my, ...","[edit, threw, out, a, diaper, beside, my, head...",https://www.reddit.com/r/AmItheAsshole/comment...,0.072667,0.834185,0.093185,0.014785
4,TYA,ebgst8,A little background. I met a guy (‘Mark’) at w...,"[A little background., I met a guy (‘Mark’) at...",WIBTA if I sent someone back their Christmas c...,"[a, little, background, ., i, met, a, guy, (, ...","[a, little, background, i, met, a, guy, mark, ...",https://www.reddit.com/r/AmItheAsshole/comment...,0.020043,0.874739,0.105174,0.184752


How do sentiment scores between the different judgemnts compare on average?

In [23]:
sentiment_scores = pd.concat(
    [pd.DataFrame(average_archive_sentiment_scores([archive]), index=[0]) for archive in archive_names.keys()],
    ignore_index=True)

archive_abr = list(archive_names.keys())
sentiment_scores.rename(index=lambda i : archive_names[archive_abr[i]], inplace=True)
sentiment_scores

,neg,neu,pos,compound
Asshole,0.073868,0.837796,0.088151,0.041086
Not the A-hole,0.078916,0.840239,0.080120,0.010996
Everyone Sucks,0.077233,0.840513,0.081234,0.004410
No A-holes here,0.070727,0.833219,0.095094,0.054878
